In [30]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [31]:
# Create path for train.csv and weather.csv

path_train = '/Users/michaelshea/desktop/class/WestNile/WestNilePrediction/Assets/train.csv'
path_weather = '/Users/michaelshea/desktop/class/WestNile/WestNilePrediction/Assets/weather.csv'

In [32]:
# Read in dataframes

train = pd.read_csv(path_train)
weather = pd.read_csv(path_weather)

In [33]:
# Dtypes and columns of train_df 

print train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object


In [34]:
# Convert train.Date to datetime

train['Date'] = pd.to_datetime(train.Date)

In [35]:
# Dtypes and columns of weather_df

print weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object


In [36]:
weather.head()

Station        Date  Tmax  Tmin Tavg Depart  DewPoint WetBulb Heat Cool  \
0        1  2007-05-01    83    50   67     14        51      56    0    2   
1        2  2007-05-01    84    52   68      M        51      57    0    3   
2        1  2007-05-02    59    42   51     -3        42      47   14    0   
3        2  2007-05-02    60    43   52      M        42      47   13    0   
4        1  2007-05-03    66    46   56      2        40      48    9    0   

     ...    CodeSum Depth Water1 SnowFall PrecipTotal StnPressure SeaLevel  \
0    ...                0      M      0.0        0.00       29.10    29.82   
1    ...                M      M        M        0.00       29.18    29.82   
2    ...         BR     0      M      0.0        0.00       29.38    30.09   
3    ...      BR HZ     M      M        M        0.00       29.44    30.08   
4    ...                0      M      0.0        0.00       29.39    30.12   

  ResultSpeed ResultDir  AvgSpeed  
0         1.7        27       9.2  
1         2.7        25       9.6  
2        13.0         4      13.4  
3        13.3         2      13.4  
4        11.7         7      11.9  

[5 rows x 22 columns]

In [37]:
train.head()

Date                                            Address  \
0 2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634,...   
1 2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634,...   
2 2007-05-29  6200 North Mandell Avenue, Chicago, IL 60646, USA   
3 2007-05-29    7900 West Foster Avenue, Chicago, IL 60656, USA   
4 2007-05-29    7900 West Foster Avenue, Chicago, IL 60656, USA   

                  Species  Block           Street  Trap  \
0  CULEX PIPIENS/RESTUANS     41   N OAK PARK AVE  T002   
1          CULEX RESTUANS     41   N OAK PARK AVE  T002   
2          CULEX RESTUANS     62    N MANDELL AVE  T007   
3  CULEX PIPIENS/RESTUANS     79     W FOSTER AVE  T015   
4          CULEX RESTUANS     79     W FOSTER AVE  T015   

              AddressNumberAndStreet   Latitude  Longitude  AddressAccuracy  \
0  4100  N OAK PARK AVE, Chicago, IL  41.954690 -87.800991                9   
1  4100  N OAK PARK AVE, Chicago, IL  41.954690 -87.800991                9   
2   6200  N MANDELL AVE, Chicago, IL  41.994991 -87.769279                9   
3    7900  W FOSTER AVE, Chicago, IL  41.974089 -87.824812                8   
4    7900  W FOSTER AVE, Chicago, IL  41.974089 -87.824812                8   

   NumMosquitos  WnvPresent  
0             1           0  
1             1           0  
2             1           0  
3             1           0  
4             4           0

In [38]:
# Convert weather Station to object

weather['Station'] = weather.Station.apply(str)

In [39]:
# Convert weather Date column to datetime

weather['Date'] = pd.to_datetime(weather.Date)

In [40]:
# Set weather index to Date

weather.set_index('Date', inplace=True)

In [41]:
# Find Date range of weather data

print weather.index.min()
print weather.index.max()

2007-05-01 00:00:00
2014-10-31 00:00:00


In [42]:
# Was weather measured every day? Every weekday?
# According to internet, there were 2741 days between the dates above, 1886 excluding weekends and public holidays

# This proves weather not measured everyday (just something to note going forward):
print len(weather.index.unique())

1472


In [43]:
# Since test set includes 2008, 2010, 2012, and 2014, pull out these years from weather

weather = weather[(weather.index.year == 2007) | (weather.index.year == 2009) | (weather.index.year == 2011) | (weather.index.year == 2013)]

In [44]:
# Demonstrate above code worked:

print np.unique(weather.index.year)

[2007 2009 2011 2013]


Since there are two weather stations, it might make sense to calculate the distance between each trap and the nearest weather station, and then use that station's weather data when we merge. The following link is what I found when I googled "calculate distance between two points latitude longitude python": http://www.johndcook.com/blog/python_longitude_latitude/

It uses this code:

In [45]:
import math
 
def distance_on_unit_sphere(lat1, long1, lat2, long2):

    # Convert latitude and longitude to spherical coordinates in radians
    degrees_to_radians = math.pi/180.0
 
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
 
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
 
    # Compute spherical distance from spherical coordinates.
 
    # For two locations in spherical coordinates
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) =
    # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
 
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
    math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )

    # Remember to multiply arc by the radius of the earth
    # in your favorite set of units to get length.
    return arc

In [46]:
# We could add a new column called closest_station to the train dataframe

station_1_lat = 41.995
station_1_lon = -87.933
station_2_lat = 41.786
station_2_lon = -87.752
dist_to_station_1 = []
dist_to_station_2 = []
closest_station = []

for i in range(len(train.index)):
    one = distance_on_unit_sphere(train.ix[i, 'Latitude'], train.ix[i, 'Longitude'], station_1_lat, station_1_lon)
    two = distance_on_unit_sphere(train.ix[i, 'Latitude'], train.ix[i, 'Longitude'], station_2_lat, station_2_lon)
    if dist_to_station_1 > dist_to_station_2:
        closest_station.append('1')
    else:
        closest_station.append('2')

In [47]:
# Now add closest_station column to weather stations to train dataframe

train['closest_station'] = closest_station

In [55]:
# Add a unique identifier column to both train and weather dataframes by combining timestamp and weather station

train['date_station_id'] = train["Date"].map(str) + train["closest_station"]

weather['date_station_id'] = weather.index.map(str) + weather.Station.map(str)

In [56]:
# Check whether unique id works

train_dates = train.date_station_id.unique()

weather_dates = set(weather.date_station_id)

for day in train_dates:
    if day in weather_dates:
        print day

2007-05-29 00:00:002
2007-06-05 00:00:002
2007-06-26 00:00:002
2007-06-29 00:00:002
2007-07-02 00:00:002
2007-07-11 00:00:002
2007-07-18 00:00:002
2007-07-19 00:00:002
2007-07-25 00:00:002
2007-07-27 00:00:002
2007-08-01 00:00:002
2007-08-02 00:00:002
2007-08-03 00:00:002
2007-08-07 00:00:002
2007-08-08 00:00:002
2007-08-09 00:00:002
2007-08-15 00:00:002
2007-08-16 00:00:002
2007-08-17 00:00:002
2007-08-21 00:00:002
2007-08-22 00:00:002
2007-08-24 00:00:002
2007-08-28 00:00:002
2007-09-04 00:00:002
2007-09-05 00:00:002
2007-09-06 00:00:002
2007-09-12 00:00:002
2007-09-18 00:00:002
2007-09-19 00:00:002
2007-09-24 00:00:002
2007-09-25 00:00:002
2007-10-04 00:00:002
2007-10-09 00:00:002
2009-05-28 00:00:002
2009-06-02 00:00:002
2009-06-03 00:00:002
2009-06-05 00:00:002
2009-06-12 00:00:002
2009-06-15 00:00:002
2009-06-19 00:00:002
2009-06-22 00:00:002
2009-06-26 00:00:002
2009-06-29 00:00:002
2009-07-06 00:00:002
2009-07-10 00:00:002
2009-07-13 00:00:002
2009-07-17 00:00:002
2009-07-24 00